<h1 style="color:red;text-align:center;">Les bibliotheques utilisées</h1>

In [4]:
import pandas as pd #Pour importer les donnees et croiser la table
import numpy as np
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


<h1 style="color:red;text-align:center;">Load dataset</h1>

In [5]:
df = pd.read_csv('./BreadBasket.csv')

<h1 style="color:red;text-align:center;">vérification des dimensions de la table</h1>

In [6]:
print("Les dimentions de la table :")
print(df.shape) #Pour verifier les dimensions de la table
print("\nLes 15 premiers lignes de la table :")
print(df.head(15)) #Pour afficher les 15 premiers lignes de la base de donnees

Les dimentions de la table :
(12935, 2)

Les 15 premiers lignes de la table :
    Transaction                      Item
0             1                   Peaches
1             2             Vegetable_Oil
2             2               Frozen_Corn
3             3                     Plums
4             4               Pancake_Mix
5             5                    Cheese
6             6               Cauliflower
7             7                 2pct_Milk
8             8  98pct_Fat_Free_Hamburger
9             8              Potato_Chips
10            8                Sesame_Oil
11            8        Ice_Cream_Sandwich
12            8       Frozen_Cheese_Pizza
13            8      Frozen_Sausage_Pizza
14            8                Deli_Salad


<h1 style="color:red;text-align:center;">Transformation en tableau binaire</h1>

In [7]:
TC = pd.crosstab(df.Transaction,df.Item) #Notre table croisee en 0/1 (1 si le item est present dans la transaction, 0 sinon)
print(TC.iloc[:25,:3]) #Affichage des 25 premiers transactions et 3 premiers items


Item         100_Watt_Lightbulb  2pct_Milk  40_Watt_Lightbulb
Transaction                                                  
1                             0          0                  0
2                             0          0                  0
3                             0          0                  0
4                             0          0                  0
5                             0          0                  0
6                             0          0                  0
7                             0          1                  0
8                             0          0                  0
9                             0          0                  0
10                            0          0                  0
11                            0          0                  0
12                            0          0                  0
13                            0          0                  0
14                            0          0                  0
15      

<h1 style="color:red;text-align:center;">La dimension du tableau TC</h1>

In [8]:
print(TC.shape)


(1360, 303)


<h1 style="color:red;text-align:center;">La liste des produits (Items)</h1>

In [9]:
print(TC.columns[:10]) #afficher les 10 premiers colonnes (items)


Index(['100_Watt_Lightbulb', '2pct_Milk', '40_Watt_Lightbulb',
       '60_Watt_Lightbulb', '75_Watt_Lightbulb', '98pct_Fat_Free_Hamburger',
       'AA_Cell_Batteries', 'Apple_Cinnamon_Waffles', 'Apple_Drink',
       'Apple_Fruit_Roll'],
      dtype='object', name='Item')


<h1 style="color:red;text-align:center;">Extraction des itemsets fréquents</h1>

In [10]:
#itemsets frequents
#support min est (0.025 x 1360), cardinale max 4.
freq_itemsets = apriori(TC,min_support=0.040,max_len=5,use_colnames=True)


In [11]:
#résultats sont stockés dans une structure de type ‘’pandas / DataFrame’’
type(freq_itemsets)
print(freq_itemsets.columns)

Index(['support', 'itemsets'], dtype='object')


In [12]:
print(freq_itemsets.shape) #Va nous permet de voir le nb d'itemsets frequents


(88, 2)


In [13]:
#affichage des 15 premiers itemsets
print(freq_itemsets.head(15))

     support                    itemsets
0   0.109559                 (2pct_Milk)
1   0.093382  (98pct_Fat_Free_Hamburger)
2   0.053676                    (Apples)
3   0.066912                   (Aspirin)
4   0.041912          (BBQ_Potato_Chips)
5   0.059559                   (Bananas)
6   0.064706                   (Bologna)
7   0.052941          (C_Cell_Batteries)
8   0.054412               (Canned_Tuna)
9   0.046324                (Cantaloupe)
10  0.077941                      (Cola)
11  0.047794            (Cottage_Cheese)
12  0.071324              (Cream_Cheese)
13  0.067647             (Domestic_Beer)
14  0.122794                      (Eggs)


<h3 style="color:blue;text-align:center;">Filtrage des itemsets fréquents</h3>

In [14]:
#type du champ '
print(type(freq_itemsets.itemsets))

<class 'pandas.core.series.Series'>


In [15]:
#accès indexé aux 5 premiers éléments des itemsets
print(freq_itemsets.itemsets[:5])

0                   (2pct_Milk)
1    (98pct_Fat_Free_Hamburger)
2                      (Apples)
3                     (Aspirin)
4            (BBQ_Potato_Chips)
Name: itemsets, dtype: object


In [16]:
#fonction de test d'inclusion
def is_inclus (x, items):
    return items.issubset(x)

In [17]:
#recherche des index des itemsets correspondant à une condition (Apples)
product = np.where(freq_itemsets.itemsets. apply (is_inclus,items={'Apples'}))
print(product)

(array([2], dtype=int64),)


In [18]:
#affichage des itemsets corresp.
print(freq_itemsets.loc[product])

    support  itemsets
2  0.053676  (Apples)


<h1 style="color:red;text-align:center;">Extraction (déduction) des règles d’association</h1>

In [19]:
#génération des règles à partir des itemsets fréquents, (fonction predefinie dans le biblio mlxtend.frequent_patterns)
regles = association_rules(freq_itemsets,metric="confidence",min_threshold=0.75)
#type de l'objet renvoyé
print(type(regles))

<class 'pandas.core.frame.DataFrame'>


In [20]:
#dimension de la table de regles qu'on a trouvé
print(regles.shape)

(0, 9)


In [21]:
#liste des colonnes
print(regles.columns)

Index(['antecedents', 'consequents', 'antecedent support',
       'consequent support', 'support', 'confidence', 'lift', 'leverage',
       'conviction'],
      dtype='object')


In [27]:
#règles en restreignant l'affichage à qqs colonnes
myRegles = regles.loc[:,['antecedents','consequents','support', 'confidence','lift']]
print(myRegles.shape)

#pour afficher toutes les colonnes
pd.set_option('display.max_columns',5)
pd.set_option('precision',3)

#affichage des 5 premières règles
print(myRegles[:5])

(0, 5)
Empty DataFrame
Columns: [antecedents, consequents, support, confidence, lift]
Index: []


In [23]:
#affichage des règles avec un LIFT supérieur ou éga l à 7
print(myRegles[myRegles['lift'].ge(7.0)])

Empty DataFrame
Columns: [antecedents, consequents, support, confidence, lift]
Index: []


In [24]:
#trier les règles dans l'ordre du lift décroissants 10 meilleurs règles
print(myRegles.sort_values(by='lift',ascending=False)[:10])

Empty DataFrame
Columns: [antecedents, consequents, support, confidence, lift]
Index: []


In [25]:
#filtrer les règles menant au conséquent White_Bread
print(myRegles[myRegles['consequents'].eq({'White_Bread'})])

Empty DataFrame
Columns: [antecedents, consequents, support, confidence, lift]
Index: []


In [26]:
#filtrer les règles contenant 'Bananas' dans leur antécédent
print(myRegles[myRegles['antecedents'].ge({'Bananas'})])

Empty DataFrame
Columns: [antecedents, consequents, support, confidence, lift]
Index: []
